<a href="https://colab.research.google.com/github/caioitalo/soulcode/blob/main/Pr%C3%A1tica_ALESP_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Resumo
- Extração, Transformação e Carregamento de dados de Dataset da Assembleia Legislativa de São Paulo. Com tratamento de dados e união de tabelas.
- Breves análises de dados


# **EXTRAÇÃO**

In [45]:
!pip install pandera

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [46]:
# Importação de bibliotecas
import pandas as pd
import numpy as np
import pandera as pa

In [47]:
# dados alocados em um bucket na Google Cloud Platform para evitar indisponibilidade
df = pd.read_csv('https://storage.googleapis.com/datalake-soulcode-ed7/arquivos%20soulcode/alesp_desp.csv')
dfbackup = df.copy()
pd.set_option('display.max_rows', 50)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# **Transformação**

In [48]:
df

,Unnamed: 0,Ano,Matricula,Mes,Valor,CNPJ,Deputado,Tipo,Fornecedor
0,0,2015,300257,3,200.00,71806251000106,ABELARDO CAMARINHA,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO DE SERVIÇOS MARV LTDA
1,1,2015,300257,3,295.40,68064740000125,ABELARDO CAMARINHA,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,PAPER FACE COMERCIAL LTDA
2,2,2015,300257,3,2850.00,22145388877,ABELARDO CAMARINHA,N - MORADIA,LARA SERVINO VARGAS ABREU
3,3,2015,300257,4,100.00,10915883000113,ABELARDO CAMARINHA,A - COMBUSTÍVEIS E LUBRIFICANTES,CENTRO AUTOMOTIVO TOP 23 LTDA
4,4,2015,300257,4,195.01,71806251000106,ABELARDO CAMARINHA,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO DE SERVIÇOS MARV LTDA
...,...,...,...,...,...,...,...,...,...
536563,536563,2007,300441,3,120.00,6306675000140.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",OPTEI EXPRESS LTDA.
536564,536564,2007,300441,3,2096.80,52204617000122.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",CHURRASCARIA JANDA LTDA.
536565,536565,2007,300441,3,13.48,62925474000282.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",ASSOCIAÇÃO FUNC.ASSEMB.LEG. EST.SÃO PAULO
536566,536566,2007,300441,3,38.08,66747577000170.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",IBIZA EUROPEIA ALIMENTOS LTDA


## Renomeando e dropando colunas desnecessárias

In [49]:
df.rename(columns={'Ano':'ano','Matricula':'matricula','Mes':'mes','Valor':'valor','CNPJ':'cnpj','Tipo':'tipo', 'Fornecedor':'fornecedor', 'Deputado':'deputado'},inplace=True)
df.drop('Unnamed: 0',axis=1, inplace=True)

## Checagens básicas de inconsistências



In [50]:
# CNPJ como objeto é algo a ser observado.

df.dtypes

ano             int64
matricula       int64
mes             int64
valor         float64
cnpj           object
deputado       object
tipo           object
fornecedor     object
dtype: object

In [51]:
# Usando unique para ver os valores únicos em cada coluna.
# pontos a observar: valores muito baixos como: 0.0; 0.1; 0.2; 0.3
# Muitos fornecedores com nomes parecidos

sorted(pd.unique(df['fornecedor']))

['" A COMADRE LANCHONETE E RESTAURANTE" PEDRO VILLELA  VILHENA - ME.',
 '" VÓ MAPHALDA "  SALADA GRILL',
 '"A TRIBUNA" DE SANTOS JORNAL E EDITORA LTDA',
 '"A VARANDA" - JOSÉ ROBERTO GERALDO - ME',
 '"ESTÂNCIA SERVIÇOS ALIMENTARES" - IVONE MEIRA DA SILVA - ME',
 '"MEGA PARK " ESTACIONAMENTO',
 '"O BRASEIRO" GALETO NA BRASA LTDA',
 '"O BRAZEIRO" GALETO NA BRASA LTDA',
 '"O REI DOS ENVELOPES" GRAF. COMERCIAL E DISTRIBUIDORA LTDA.',
 '"POSTO SÃO SEBASTIÃO" - AUTO POSTO CESTO GARCIA LTDA',
 "'FATOR GRAFICO GRAFICA E EDITORA LTDA -ME",
 "'FREIRE VEÍCULOS",
 '(POSTO MENEGUETTI) AUTO POSTO JOTAVE LTDA',
 ',ARIA HELENA GOMES RESTAURANTE ME',
 ',ARIA JULIA BARNABE BARBOZA - ME',
 ',AUTO POSTO CENTER LESTE LTDA',
 ',AUTO POSTO ITAPURA',
 ',FENIX AUTO POSTO DE SERVIÇOS LTDA',
 ',ODELO PAPELARIA E LIVRARIA LTDA',
 ',POSTO DE SERVIÇOS MARIANA LTDA',
 ",POSTO DE SERVIÇOS PAP'S ENSEADA LTDA",
 ',RONER PIRES NUNES - ME',
 '-AUTO POSTO MOEMA DA FRONTEIRA LTDA',
 '.K.F. COMÉRCIO DE ALIMENTOS LTDA',
 ".MC

In [52]:
# Checagem de valores NaN
df.isna().sum()

ano               0
matricula         0
mes               0
valor             0
cnpj          10892
deputado          0
tipo              0
fornecedor        0
dtype: int64

## TRATAMENTO BÁSICO DA BD

In [53]:
sorted(pd.unique(df['fornecedor']))

['" A COMADRE LANCHONETE E RESTAURANTE" PEDRO VILLELA  VILHENA - ME.',
 '" VÓ MAPHALDA "  SALADA GRILL',
 '"A TRIBUNA" DE SANTOS JORNAL E EDITORA LTDA',
 '"A VARANDA" - JOSÉ ROBERTO GERALDO - ME',
 '"ESTÂNCIA SERVIÇOS ALIMENTARES" - IVONE MEIRA DA SILVA - ME',
 '"MEGA PARK " ESTACIONAMENTO',
 '"O BRASEIRO" GALETO NA BRASA LTDA',
 '"O BRAZEIRO" GALETO NA BRASA LTDA',
 '"O REI DOS ENVELOPES" GRAF. COMERCIAL E DISTRIBUIDORA LTDA.',
 '"POSTO SÃO SEBASTIÃO" - AUTO POSTO CESTO GARCIA LTDA',
 "'FATOR GRAFICO GRAFICA E EDITORA LTDA -ME",
 "'FREIRE VEÍCULOS",
 '(POSTO MENEGUETTI) AUTO POSTO JOTAVE LTDA',
 ',ARIA HELENA GOMES RESTAURANTE ME',
 ',ARIA JULIA BARNABE BARBOZA - ME',
 ',AUTO POSTO CENTER LESTE LTDA',
 ',AUTO POSTO ITAPURA',
 ',FENIX AUTO POSTO DE SERVIÇOS LTDA',
 ',ODELO PAPELARIA E LIVRARIA LTDA',
 ',POSTO DE SERVIÇOS MARIANA LTDA',
 ",POSTO DE SERVIÇOS PAP'S ENSEADA LTDA",
 ',RONER PIRES NUNES - ME',
 '-AUTO POSTO MOEMA DA FRONTEIRA LTDA',
 '.K.F. COMÉRCIO DE ALIMENTOS LTDA',
 ".MC

In [54]:
df.dtypes

ano             int64
matricula       int64
mes             int64
valor         float64
cnpj           object
deputado       object
tipo           object
fornecedor     object
dtype: object

### O problema dos CNPJs

In [55]:
# verificando se todos os CNPJs são numéricos, percebe-se que não.
df.cnpj.astype(float)

ValueError: ignored

In [56]:
# Coletando os valores únicos de CNPJ
dfun = df.cnpj.unique()
dfun

array(['71806251000106', '68064740000125', '22145388877', ...,
       50323741991.0, 6878968000100.0, 5591224000139.0], dtype=object)

In [57]:
# Percorrendo os valores únicos para encontrar os cnpjs que não são númericos
valores_estranhos = []
for i in range(len(dfun)):
  try:
    int(dfun[i])
  except Exception:
    valores_estranhos.append(dfun[i])
print(valores_estranhos)

[nan, 'O4689799000126', 'O4040553000129', 'O2358317000120', 'O1520430000105', '00O21449000136', 'O7482309000104']


In [58]:
# todos os casos em que uma letra 'O' foi colocado em vez do 0
ftcnpj = df[df.cnpj.isin(['O4689799000126','O4040553000129','O2358317000120','O1520430000105','00O21449000136','O7482309000104'])]
ftcnpj

,ano,matricula,mes,valor,cnpj,deputado,tipo,fornecedor
55891,2007,300482,11,76.78,O4689799000126,ANTÔNIO CARLOS,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",VIA BARBARESCO LTDA
55964,2008,300482,2,70.02,O4040553000129,ANTÔNIO CARLOS,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO AUSTRIA LTDA
55998,2008,300482,3,50.00,O4040553000129,ANTÔNIO CARLOS,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO AUSTRIA LTDA
316251,2007,300455,9,40.00,O2358317000120,MARCOS MARTINS,"G - ASSINATURAS DE PERIÓDICOS, PUBLICAÇÕES, IN...",BASENET INFORMATICA LTDA
365639,2007,300468,12,504.12,O1520430000105,OLÍMPIO GOMES,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",PICANHARIA DO GAUCHO/ JOCELI NARDI CHURRASCARI...
389510,2002,300379,5,786.55,00O21449000136,PEDRO YVES,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO BOSQUE SATÉLITE LTDA
389553,2002,300379,9,915.76,00O21449000136,PEDRO YVES,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO BOSQUE SATÉLITE LTDA
389565,2002,300379,10,802.69,00O21449000136,PEDRO YVES,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO BOSQUE SATÉLITE LTDA
389578,2002,300379,12,3101.63,00O21449000136,PEDRO YVES,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO BOSQUE SATÉLITE LTDA
389582,2003,300379,1,5472.30,00O21449000136,PEDRO YVES,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO BOSQUE SATÉLITE LTDA


In [59]:
# Trocando o erro, muito provavelmente por digitações erradas em sistemas antigos
df2 = df.copy()
df2.cnpj.replace('O','0', regex=True, inplace=True)
df2

,ano,matricula,mes,valor,cnpj,deputado,tipo,fornecedor
0,2015,300257,3,200.00,71806251000106,ABELARDO CAMARINHA,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO DE SERVIÇOS MARV LTDA
1,2015,300257,3,295.40,68064740000125,ABELARDO CAMARINHA,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,PAPER FACE COMERCIAL LTDA
2,2015,300257,3,2850.00,22145388877,ABELARDO CAMARINHA,N - MORADIA,LARA SERVINO VARGAS ABREU
3,2015,300257,4,100.00,10915883000113,ABELARDO CAMARINHA,A - COMBUSTÍVEIS E LUBRIFICANTES,CENTRO AUTOMOTIVO TOP 23 LTDA
4,2015,300257,4,195.01,71806251000106,ABELARDO CAMARINHA,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO DE SERVIÇOS MARV LTDA
...,...,...,...,...,...,...,...,...
536563,2007,300441,3,120.00,6306675000140.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",OPTEI EXPRESS LTDA.
536564,2007,300441,3,2096.80,52204617000122.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",CHURRASCARIA JANDA LTDA.
536565,2007,300441,3,13.48,62925474000282.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",ASSOCIAÇÃO FUNC.ASSEMB.LEG. EST.SÃO PAULO
536566,2007,300441,3,38.08,66747577000170.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",IBIZA EUROPEIA ALIMENTOS LTDA


In [60]:
# Agrupando e mostrando os casos em que há valores vazios para possíveis inconsistências.
# Nota-se vários campos com nomes similares
dfcnpjnan = df[df.cnpj.isna()]

In [61]:
dfcnpjnan.groupby(['fornecedor']).size()

fornecedor
ADOBE SYSTEMS SOFTWARE IRELAND LTD                   1
ADOBE SYSTEMS SOFTWARE IRELAND LTDA                  5
CONCESSIONARIA RODOVIA OESTE DE SAO PAULO S.A.       1
CUPONS DE PEDÁGIO                                    1
DELTA RADIO TAXI                                     1
DROPBOX INTERNACIONAL UNLIMETED COMPANY              1
ELOY DE PAULA JUNIOR                                 1
FLICKR, INC                                          1
GOOGLE ONE                                           2
GOOGLE STORAGE - GOOGLE ONE                          1
LEANDRO MATOS NICOLELA                               1
PEDAGIO                                             11
PEDAGIOS                                             1
PEDÁGIO                                           7476
PEDÁGIOS                                             2
PEGAGIO                                              1
TAXI                                              3385
dtype: int64

In [62]:
# Fazendo o replace dessas inconsistências
df2.fornecedor.replace({'ADOBE SYSTEMS SOFTWARE IRELAND LTD':'ADOBE SYSTEMS SOFTWARE IRELAND LTDA',
                              'PEDAGIOS':'PEDÁGIO','PEDAGIO':'PEDÁGIO','PEDÁGIOS':'PEDÁGIO','PEGAGIO':'PEDÁGIO','CUPONS DE PEDÁGIO':'PEDÁGIO',
                               'GOOGLE STORAGE - GOOGLE ONE':'GOOGLE ONE'},inplace=True)


In [63]:
# Criando um schema de validação de dados
schema = pa.DataFrameSchema(
  columns={
    'ano':pa.Column(pa.Int),
    'matricula':pa.Column(pa.Int),
    'mes':pa.Column(pa.Int),
    'valor':pa.Column(pa.Float),
    'cnpj':pa.Column(pa.String,nullable=True),
    'tipo':pa.Column(pa.String),
    'fornecedor':pa.Column(pa.String),
    'deputado':pa.Column(pa.String)
    }
)

In [64]:
# validando o DF
schema.validate(df2)

,ano,matricula,mes,valor,cnpj,deputado,tipo,fornecedor
0,2015,300257,3,200.00,71806251000106,ABELARDO CAMARINHA,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO DE SERVIÇOS MARV LTDA
1,2015,300257,3,295.40,68064740000125,ABELARDO CAMARINHA,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,PAPER FACE COMERCIAL LTDA
2,2015,300257,3,2850.00,22145388877,ABELARDO CAMARINHA,N - MORADIA,LARA SERVINO VARGAS ABREU
3,2015,300257,4,100.00,10915883000113,ABELARDO CAMARINHA,A - COMBUSTÍVEIS E LUBRIFICANTES,CENTRO AUTOMOTIVO TOP 23 LTDA
4,2015,300257,4,195.01,71806251000106,ABELARDO CAMARINHA,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO DE SERVIÇOS MARV LTDA
...,...,...,...,...,...,...,...,...
536563,2007,300441,3,120.00,6306675000140.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",OPTEI EXPRESS LTDA.
536564,2007,300441,3,2096.80,52204617000122.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",CHURRASCARIA JANDA LTDA.
536565,2007,300441,3,13.48,62925474000282.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",ASSOCIAÇÃO FUNC.ASSEMB.LEG. EST.SÃO PAULO
536566,2007,300441,3,38.08,66747577000170.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",IBIZA EUROPEIA ALIMENTOS LTDA


In [65]:
# pegando os dados da última legislatura
dfleg = df[(df.ano > 2018) & (df.ano < 2023)]
dfleg

,ano,matricula,mes,valor,cnpj,deputado,tipo,fornecedor
293,2019,300257,1,8000.00,03136828000160,ABELARDO CAMARINHA,"D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS ...",MARCO ANTONIO D AVILA
294,2019,300257,1,5600.00,06222505000187,ABELARDO CAMARINHA,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,WAGNER HUBERT EPP
295,2019,300257,1,5500.00,17035632000193,ABELARDO CAMARINHA,F - SERVIÇOS TÉCNICOS PROFISSIONAIS (CONSULTOR...,RED VISION PRODUTORA LTDA-ME
296,2019,300257,1,8000.00,11827486000152,ABELARDO CAMARINHA,J - SERVIÇOS DE COMUNICAÇÃO,IZABEL DE FATIMA RAMOS PEREIRA
297,2019,300257,1,1995.00,06222505000187,ABELARDO CAMARINHA,K - LOCAÇÃO DE BENS MÓVEIS,WAGNER HUBERT EPP
...,...,...,...,...,...,...,...,...
533820,2019,300523,3,60.71,4353528815.0,WELSON GASPARINI,"M - MANUTENÇÃO DE BENS MÓVEIS, IMÓVEIS, CONDO...",NEIDE GONDIM BORGES LUIZ
533821,2019,300523,3,114.34,9621029000165.0,WELSON GASPARINI,"M - MANUTENÇÃO DE BENS MÓVEIS, IMÓVEIS, CONDO...",MW5 INFORMATICA E SERVIÇOS LTDA. ME
533822,2019,300523,3,163.33,31112521000164.0,WELSON GASPARINI,"M - MANUTENÇÃO DE BENS MÓVEIS, IMÓVEIS, CONDO...",QUATRO TECH SERVIÇOS E TECNOLOGIA
533823,2019,300523,3,72.83,61216917000112.0,WELSON GASPARINI,"M - MANUTENÇÃO DE BENS MÓVEIS, IMÓVEIS, CONDO...",CENTRO ELETRONICO E COMERCIAL SETRON LTDA. - EPP


#**CARREGAMENTO DOS DADOS**

In [43]:
df2.to_csv('ALESP_Desp_Final.csv',index=False)

#**ANÁLISES BÁSICAS DOS DADOS**